<a href="https://colab.research.google.com/github/NateMophi/SCC-454/blob/main/LAB4/SCC454_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install required packages
!pip install pyspark==3.5.0 -q
!pip install sentence-transformers -q
!pip install numpy pandas -q

# Install Java (Spark requires Java)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Set Java environment variable
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

print("All packages installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 1.0.1 requires pyspark[connect]~=4.0.0, but you have pyspark 3.5.0 which is incompatible.
All packages installed successfully!


In [4]:
import numpy as np
import hashlib
from typing import List, Set, Tuple

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import(
    col, udf, explode, array, lit, collect_list, size, lower, regexp_replace, split, monotonically_increasing_id, struct,
    when, coalesce, broadcast
)

from pyspark.sql.types import(
    ArrayType, StringType, IntegerType, FloatType, StructType, StructField,
    DoubleType
)

from pyspark.ml.feature import (
    HashingTF, CountVectorizer, MinHashLSH,
    Tokenizer, StopWordsRemover, NGram
)

from pyspark.ml.linalg import Vectors, SparseVector, VectorUDT
from pyspark.ml import Pipeline



In [6]:
# Spark Session for LSH Operations
spark = SparkSession.builder.appName("SCC454-LocalitySensitiveHashing")\
.config("spark.driver.memory", "4g")\
.config("spark.sql.shuffle.partitions", "8")\
.config("spark.ui.port", "4050")\
.getOrCreate()

sc = spark.sparkContext
print(f"Spark Version: {spark.version}")
print(f"App Name: {spark.sparkContext.appName}")
print("\nSpark Session Ready for LSH ops!")

Spark Version: 3.5.0
App Name: SCC454-LocalitySensitiveHashing

Spark Session Ready for LSH ops!


## SAMPLE DATASET

In [7]:
# Sample document corpus with varying degrees of similarity
documents = [
    (0, "Machine learning is a subset of artificial intelligence that enables systems to learn from data."),
    (1, "Artificial intelligence and machine learning allow computers to learn from data automatically."),
    (2, "Deep learning is a type of machine learning using neural networks with many layers."),
    (3, "The weather today is sunny with a high of 25 degrees celsius."),
    (4, "Today's weather forecast shows sunny skies and temperatures around 25 degrees."),
    (5, "Natural language processing helps computers understand human language."),
    (6, "NLP enables machines to process and understand natural human language."),
    (7, "Python is a popular programming language for data science and machine learning."),
    (8, "Data science often uses Python programming for machine learning applications."),
    (9, "The cat sat on the mat and watched the birds outside the window."),
    (10, "A small cat was sitting on a mat, watching birds through the window."),
    (11, "Apache Spark provides distributed computing for big data processing."),
    (12, "Big data processing is made efficient through distributed computing with Spark."),
    (13, "Locality sensitive hashing enables fast approximate nearest neighbor search."),
    (14, "LSH provides fast approximate nearest neighbor queries using hashing techniques."),
    (15, "The restaurant serves delicious Italian pasta and fresh salads daily."),
]

df_docs = spark.createDataFrame(documents, ["id", "text"])

print("Sample Document Corpus:")
df_docs.show(truncate=60)

Sample Document Corpus:
+---+------------------------------------------------------------+
| id|                                                        text|
+---+------------------------------------------------------------+
|  0|Machine learning is a subset of artificial intelligence t...|
|  1|Artificial intelligence and machine learning allow comput...|
|  2|Deep learning is a type of machine learning using neural ...|
|  3|The weather today is sunny with a high of 25 degrees cels...|
|  4|Today's weather forecast shows sunny skies and temperatur...|
|  5|Natural language processing helps computers understand hu...|
|  6|NLP enables machines to process and understand natural hu...|
|  7|Python is a popular programming language for data science...|
|  8|Data science often uses Python programming for machine le...|
|  9|The cat sat on the mat and watched the birds outside the ...|
| 10|A small cat was sitting on a mat, watching birds through ...|
| 11|Apache Spark provides distributed

## **DOCUMENT SHINGLING**

---
# Part 2: Document Shingling
---

## 2.1 Understanding Shingling

**What is Shingling?**
Shingling converts documents into sets of contiguous subsequences (shingles). This allows us to measure document similarity by comparing these sets.

**Types of Shingles:**

| Type | Description | Example ("hello world") |
|------|-------------|-------------------------|
| Character k-shingles | Contiguous k characters | {"hel", "ell", "llo", "lo ", "o w", ...} |
| Word n-grams | Contiguous n words | {"hello world"} for n=2 |

**Choosing Shingle Size:**
- Too small: High overlap even for dissimilar documents
- Too large: Low overlap even for similar documents
- Rule of thumb: k=5-9 for characters, n=2-4 for words

In [8]:
# CHARACTER SHINGLES
def get_char_shingles(text:str, k:int =5):
  """Generate character k-shingles from text"""
  text = " ".join(text.lower().split())

  # Generate Shingles
  shingles = [text[i:i+k] for i in range(len(text) - k + 1)]
  return shingles

# Example
sample_text = "Hello World"
char_shingles = get_char_shingles(sample_text, k=5)
print(f"Text: '{sample_text}'")
print(f"5-character shingles: {char_shingles}")
print(f"Number of shingles: {len(char_shingles)}")
print(f"Unique shingles: {len(set(char_shingles))}")

Text: 'Hello World'
5-character shingles: ['hello', 'ello ', 'llo w', 'lo wo', 'o wor', ' worl', 'world']
Number of shingles: 7
Unique shingles: 7


In [9]:
# WORD SHINGLES (n-grams)
def get_word_shingles(text: str, n:int=3)-> List[str]:
  words = text.lower().split()
  shingles = [" ".join(words[i:i+n]) for i in range(len(words) - n + 1)]
  return shingles

  # Example
sample_text = "Machine learning is a subset of artificial intelligence"
word_shingles = get_word_shingles(sample_text, n=3)
print(f"Text: '{sample_text}'")
print(f"\n3-word shingles:")
for i, shingle in enumerate(word_shingles):
    print(f"  {i+1}. '{shingle}'")

Text: 'Machine learning is a subset of artificial intelligence'

3-word shingles:
  1. 'machine learning is'
  2. 'learning is a'
  3. 'is a subset'
  4. 'a subset of'
  5. 'subset of artificial'
  6. 'of artificial intelligence'


In [10]:
# Shingling in SPARK
# Method 1: Using Spark's NGram transformer
def _create_shingle_pipeline(n_values=[2, 3, 4]):
  """Create a pipeline for generating word n-grams of multiple sizes """
  from pyspark.sql.functions import concat_ws, flatten

  # Tokenizer
  tokenizer = Tokenizer(inputCol="text_clean", outputCol="words")

  # Create NGram transformers for each n
  ngram_transformers =  []
  for n in n_values:
    ngram = NGram(n=n, inputCol="words", outputCol=f"ngrams_{n}")
    ngram_transformers.append(ngram)

  return tokenizer, ngram_transformers

# Preprocessing Text

df_clean = df_docs.withColumn(
    "text_clean",
    lower(regexp_replace(col("text"), r"[^a-zA-Z\s]", ""))
)

# Apply Tokenizer
tokenizer = Tokenizer(inputCol="text_clean", outputCol="words")
df_tokenized = tokenizer.transform(df_clean)

# Applying n-grams transforemers for n = 2,3,4
ngram_2 = NGram(n=2, inputCol="words", outputCol="ngrams_2")
ngram_3 = NGram(n=3, inputCol="words", outputCol="ngrams_3")
ngram_4 = NGram(n=4, inputCol="words", outputCol="ngrams_4")

df_shingles = ngram_2.transform(df_tokenized)
df_shingles = ngram_3.transform(df_shingles)
df_shingles = ngram_4.transform(df_shingles)

print("Document with word shingles (n=2,3,4):")
df_shingles.select("id", "ngrams_2", "ngrams_3").show(5, truncate=50)

Document with word shingles (n=2,3,4):
+---+--------------------------------------------------+--------------------------------------------------+
| id|                                          ngrams_2|                                          ngrams_3|
+---+--------------------------------------------------+--------------------------------------------------+
|  0|[machine learning, learning is, is a, a subset,...|[machine learning is, learning is a, is a subse...|
|  1|[artificial intelligence, intelligence and, and...|[artificial intelligence and, intelligence and ...|
|  2|[deep learning, learning is, is a, a type, type...|[deep learning is, learning is a, is a type, a ...|
|  3|[the weather, weather today, today is, is sunny...|[the weather today, weather today is, today is ...|
|  4|[todays weather, weather forecast, forecast sho...|[todays weather forecast, weather forecast show...|
+---+--------------------------------------------------+-----------------------------------------

In [11]:
# Method 2: Combine all n-grams into a single sheet
from pyspark.sql.functions import concat, array_union, array_distinct

# Combine 2-grams, 3-grams and 4-grams into a single shingle set
df_combined_shingles = df_shingles.withColumn(
    "all_shingles",
    array_distinct(
        concat(col("ngrams_2"), col("ngrams_3"), col("ngrams_4"))
    )
)

print("Combined Shingless (2-4 grams):")
df_combined_shingles.select("id", "all_shingles").show(3, truncate=80)

Combined Shingless (2-4 grams):
+---+--------------------------------------------------------------------------------+
| id|                                                                    all_shingles|
+---+--------------------------------------------------------------------------------+
|  0|[machine learning, learning is, is a, a subset, subset of, of artificial, art...|
|  1|[artificial intelligence, intelligence and, and machine, machine learning, le...|
|  2|[deep learning, learning is, is a, a type, type of, of machine, machine learn...|
+---+--------------------------------------------------------------------------------+
only showing top 3 rows



In [12]:
# Jaccard similarity from shingle sets
def jaccard_similarity(set1: set, set2: set) -> float:
    """Calculate Jaccard similarity between two sets."""
    if not set1 or not set2:
        return 0.0
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0.0

    # Get shingles for comparison
shingle_data = df_combined_shingles.select("id", "text", "all_shingles").collect()

# Compare a few document pairs
pairs_to_compare = [(0, 1), (0, 2), (0, 3), (3, 4), (9, 10), (0, 15)]

print("Jaccard Similarity Analysis:")
print("=" * 70)

for i, j in pairs_to_compare:
    doc_i = next(row for row in shingle_data if row.id == i)
    doc_j = next(row for row in shingle_data if row.id == j)

    set_i = set(doc_i.all_shingles)
    set_j = set(doc_j.all_shingles)

    jaccard = jaccard_similarity(set_i, set_j)

    print(f"\nDocs {i} vs {j}: Jaccard = {jaccard:.4f}")
    print(f"  Doc {i}: {doc_i.text[:60]}...")
    print(f"  Doc {j}: {doc_j.text[:60]}...")
    print(f"  Shingles: {len(set_i)} vs {len(set_j)}, Overlap: {len(set_i & set_j)}")

Jaccard Similarity Analysis:

Docs 0 vs 1: Jaccard = 0.1311
  Doc 0: Machine learning is a subset of artificial intelligence that...
  Doc 1: Artificial intelligence and machine learning allow computers...
  Shingles: 39 vs 30, Overlap: 8

Docs 0 vs 2: Jaccard = 0.0563
  Doc 0: Machine learning is a subset of artificial intelligence that...
  Doc 2: Deep learning is a type of machine learning using neural net...
  Shingles: 39 vs 36, Overlap: 4

Docs 0 vs 3: Jaccard = 0.0000
  Doc 0: Machine learning is a subset of artificial intelligence that...
  Doc 3: The weather today is sunny with a high of 25 degrees celsius...
  Shingles: 39 vs 30, Overlap: 0

Docs 3 vs 4: Jaccard = 0.0179
  Doc 3: The weather today is sunny with a high of 25 degrees celsius...
  Doc 4: Today's weather forecast shows sunny skies and temperatures ...
  Shingles: 30 vs 27, Overlap: 1

Docs 9 vs 10: Jaccard = 0.0154
  Doc 9: The cat sat on the mat and watched the birds outside the win...
  Doc 10: A small cat was 

---
# Part 3: MinHash Fundamentals
---

## 3.1 The Jaccard Similarity Problem

Computing Jaccard similarity directly has limitations:
- Requires storing and comparing full sets
- Memory-intensive for large vocabularies
- O(n) per comparison where n is set size

**MinHash Solution:**
Create compact "signatures" that preserve similarity information.

## 3.2 The MinHash Algorithm

**Key Insight:**
For any random permutation π of the universal set:
```
P[min(π(A)) = min(π(B))] = Jaccard(A, B)
```

**Algorithm:**
1. Choose k random hash functions (simulating permutations)
2. For each document, compute k minimum hash values
3. The k values form the "MinHash signature"
4. Signature similarity estimates Jaccard similarity

In [12]:
class MinHasher:

  def __init__(self, num_hashes:int= 100, seed:int= 42):
    """
    Initialize MinHasher with num_hashes hash functions.
    Args:
      num_hashes: Number of hash functions (signature length)
      seed: Random seed for reproducibility
      """
    self.num_hashes =  num_hashes
    np.random.seed(seed)


    # Gen random coeffs for # function
    # h(x) = (a*x + b) mod c

    self.max_hash = 2**32 - 1
    self.a = np.random.randint(1, self.max_hashm, size=num_hashes)
    self.b = np.random.randint(0, self.max_hash, size=num_hashes)
    self.c = 4294967311

  def _hash_element(self, element:str)-> int:
    """Hash a string element to an integer."""
    return int(hashlib.md5(element.encode()).hexdigest(), 16) % self.max_hash

  def get_signature(self, shingle_set: set) -> np.ndarray:
        """
        Generate MinHash signature for a set of shingles.

        Args:
            shingle_set: Set of string shingles

        Returns:
            MinHash signature array of length num_hashes
        """
        # Initialize signature with infinity
        signature = np.full(self.num_hashes, np.inf)

        for shingle in shingle_set:
            # Hash the shingle to an integer
            shingle_hash = self._hash_element(shingle)

            # Apply all hash functions and keep minimum
            hash_values = (self.a * shingle_hash + self.b) % self.c
            signature = np.minimum(signature, hash_values)

        return signature.astype(np.int64)

    def estimate_similarity(self, sig1: np.ndarray, sig2: np.ndarray) -> float:
        """
        Estimate Jaccard similarity from MinHash signatures.

        Returns:
            Estimated Jaccard similarity
        """
        return np.mean(sig1 == sig2)



